In [1]:
# Code für NEAT KI

import pygame
import neat
import random

pygame-ce 2.3.2 (SDL 2.26.5, Python 3.7.9)


ModuleNotFoundError: No module named 'neat'

In [ ]:
pip install pygame-ce

In [ ]:
# Konfigurations Datei erstellen

config = """
[NEAT]
fitness_criterion     = max
fitness_threshold     = 100000000
pop_size              = 100
reset_on_extinction   = True

[DefaultGenome]
# node activation options
activation_default      = tanh
activation_mutate_rate  = 0.01
activation_options      = tanh

# node aggregation options
aggregation_default     = sum
aggregation_mutate_rate = 0.01
aggregation_options     = sum

# node bias options
bias_init_mean          = 0.0
bias_init_stdev         = 1.0
bias_max_value          = 30.0
bias_min_value          = -30.0
bias_mutate_power       = 0.5
bias_mutate_rate        = 0.7
bias_replace_rate       = 0.1

# genome compatibility options
compatibility_disjoint_coefficient = 1.0
compatibility_weight_coefficient   = 0.5

# connection add/remove rates
conn_add_prob           = 0.5
conn_delete_prob        = 0.5

# connection enable options
enabled_default         = True
enabled_mutate_rate     = 0.01

feed_forward            = True
initial_connection      = full

# node add/remove rates
node_add_prob           = 0.2
node_delete_prob        = 0.2

# network parameters
num_hidden              = 0
num_inputs              = 6
num_outputs             = 2

# node response options
response_init_mean      = 1.0
response_init_stdev     = 0.0
response_max_value      = 30.0
response_min_value      = -30.0
response_mutate_power   = 0.0
response_mutate_rate    = 0.0
response_replace_rate   = 0.0

# connection weight options
weight_init_mean        = 0.0
weight_init_stdev       = 1.0
weight_max_value        = 30
weight_min_value        = -30
weight_mutate_power     = 0.5
weight_mutate_rate      = 0.8
weight_replace_rate     = 0.1

[DefaultSpeciesSet]
compatibility_threshold = 2.0

[DefaultStagnation]
species_fitness_func = max
max_stagnation       = 20
species_elitism      = 2

[DefaultReproduction]
elitism            = 3
survival_threshold = 0.2
"""

config_file = open("config.txt", "w")
config_file.write(config)
config_file.close()

In [ ]:
# Code zum Erstellen von Irrgarten Sturkturen / Bauplan des Irrgarten als Matrix dargestellt


WALL = 1
GOAL = 2

class MazeGenerator:
    @staticmethod
    def generate_maze(size):
        """
        Erstellt ein Irrgarten der angegebenen Größe und gibt es als 2D-Liste zurück.
        """
        grid = MazeGenerator.create_empty_grid(size)
        MazeGenerator.generate_paths(grid, size)
        maze = MazeGenerator.transform_maze(grid, size)
        maze = MazeGenerator.add_borders(maze, size)
        return maze

    @staticmethod
    def create_empty_grid(size):
        """
        Erstellt ein leeres Gitter für das Labyrinth, wobei jede Zelle drei Werte enthält:
        [Ostwand (1 = vorhanden), Südwand (1 = vorhanden), besucht (0 = unbesucht, 1 = besucht)].
        """
        cell_template = [1, 1, 0]  # Standardwerte für jede Zelle
        grid = []
        for _ in range(size):
            row = []
            for _ in range(size):
                row.append(cell_template.copy())
            grid.append(row)
        return grid

    @staticmethod
    def generate_paths(grid, size):
        """
        Erzeugt zufällige Pfade im Labyrinth mit dem Tiefensuchalgorithmus (DFS).
        """
        stack = [(0, 0)]  # Startpunkt (oben links)
        visited_cells = 1
        total_cells = pow(size, 2)
        grid[0][0][2] = 1  # Markiere Startzelle als besucht
        while visited_cells < total_cells:
            x, y = stack[-1]  # Letzte Position im Stack
            neighbors = []

            # Prüfe alle möglichen Nachbarn
            if y > 0 and not grid[y - 1][x][2]:  # Nord
                neighbors.append(0)

            if y < size - 1 and not grid[y + 1][x][2]:  # Süd
                neighbors.append(1)

            if x > 0 and not grid[y][x - 1][2]:  # West
                neighbors.append(2)

            if x < size - 1 and not grid[y][x + 1][2]:  # Ost
                neighbors.append(3)

            if len(neighbors) == 0:
                stack.pop()  # Kein unbesuchter Nachbar -> Zurückgehen
                continue

            rand_i = random.randrange(0, len(neighbors))

            match neighbors[rand_i]:
                case 0:  # Nord
                    stack.append((x, y - 1))
                    grid[y - 1][x][1] = 0  # Entferne Südwand der neuen Zelle
                    grid[y - 1][x][2] = 1  # Markiere als besucht
                case 1:  # Süd
                    stack.append((x, y + 1))
                    grid[y][x][1] = 0  # Entferne Südwand der aktuellen Zelle
                    grid[y + 1][x][2] = 1
                case 2:  # West
                    stack.append((x - 1, y))
                    grid[y][x - 1][0] = 0  # Entferne Ostwand der neuen Zelle
                    grid[y][x - 1][2] = 1
                case 3:  # East
                    stack.append((x + 1, y))
                    grid[y][x][0] = 0  # Entferne Ostwand der aktuellen Zelle
                    grid[y][x + 1][2] = 1

            visited_cells += 1

    @staticmethod
    def transform_maze(grid, size):
        """
        Wandelt das Irrgarten Gitter in eine Matrix um. Eine Zelle ist entweder ein Ziel, eine Wand oder lehr.
        """
        maze_size = size * 2 - 1
        maze = []
        for _ in range(maze_size):
            row = [0] * (maze_size)
            maze.append(row)
        for row_index, row in enumerate(grid):
            for col_index, cell in enumerate(row):
                if (2 * col_index + 1) < maze_size and cell[0]:  # Ostwand
                    maze[2 * row_index][2 * col_index + 1] = WALL
                if (2 * row_index + 1) < maze_size and cell[1]:  # Südwand
                    maze[2 * row_index + 1][2 * col_index] = WALL
                if (2 * row_index + 1) < maze_size and (2 * col_index + 1) < maze_size:
                    maze[2 * row_index + 1][2 * col_index + 1] = WALL  # Eckpunkte

        maze[-1][-1] = GOAL # Setz das Ziel
        return maze

    @staticmethod
    def add_borders(maze, maze_size):
        """
        Fügt eine durchgehende äußere Wand zum Irrgarten hinzu.
        """
        final_size = maze_size * 2 + 1
        for maze_row in maze:
            maze_row.insert(0, 1)  # Links Wand hinzufügen
            maze_row.append(1)  # Rechts Wand hinzufügen
        maze.insert(0, [1] * final_size)  # Oben Wand hinzufügen
        maze.append([1] * final_size)  # Unten Wand hinzufügen
        return maze

In [ ]:
# Code zum erstellen von Irrgarten Objeckten (Mit Rendering)


class MazeRendererWithCollision:
    def __init__(self, maze_size: int, cell_width: int):
        """
        Initialisiert den Renderer und Kollisionslogik für das Labyrinth.
        """
        self.maze = MazeGenerator.generate_maze(maze_size)
        self.cell_width = cell_width
        maze_surface_size = len(self.maze) * cell_width # Gesamtgröße des Labyrinths in Pixeln
        self.image = pygame.Surface((maze_surface_size, maze_surface_size)) # Erstelle eine Oberfläche für das Labyrinth
        self.rect = self.image.get_rect() # Rechteck für das Labyrinth
        self.boxes: list[pygame.Rect] = []  # Liste zur Speicherung der Rechtecke für Wände und Wege und für Kollision
        self.boxes_type: list[bool] = [] # Liste, die angibt, ob eine Kollision Zelle ein Ziel ist
        self.path_cells: list[pygame.Rect] = [] # Liste der Zellen, die als Pfad markiert sind
        self.setup()

    def setup(self):
        """
        Rendert das Labyrinth auf die Oberfläche und speichert die Positionen der Zellen für Kollisionszwecke.
        """
        maze_cell = pygame.Surface((self.cell_width, self.cell_width))
        maze_cell.fill("Black")
        goal_cell = maze_cell.copy()
        goal_cell.fill("Blue")
        self.image.fill("White")

        current_rect = maze_cell.get_rect()

        for maze_row in self.maze:
            current_rect.x = 0
            for maze_block in maze_row:
                if maze_block == WALL:
                    wall = current_rect.copy()
                    self.image.blit(maze_cell, wall)
                    self.boxes.append(wall)
                    self.boxes_type.append(False)
                elif maze_block == GOAL:
                    wall = current_rect.copy()
                    self.image.blit(goal_cell, current_rect)
                    self.boxes.append(wall)
                    self.boxes_type.append(True)
                else:
                    path_cell = current_rect.copy()
                    self.path_cells.append(path_cell)
                current_rect.x += self.cell_width
            current_rect.y += self.cell_width

        self.draw_grid_lines()

    def draw_grid_lines(self):
        """
        Zeichnet die Gitterlinien für das Labyrinth (die Trennlinien zwischen den Zellen).
        """
        for i in range(len(self.maze) + 1):
            pygame.draw.line(
                self.image,
                "Gray",
                (0, self.cell_width * i),
                (self.image.width, self.cell_width * i),
                2,
            )  # Vertikale Linie
            pygame.draw.line(
                self.image,
                "Gray",
                (self.cell_width * i, 0),
                (self.cell_width * i, self.image.width),
                2,
            )  # Horizontale Linie

    def draw(self, screen: pygame.Surface):
        """
        Zeichnet das Labyrinth auf den angegebenen Bildschirm.
        """
        screen.blit(self.image, self.rect)



In [ ]:
# Code für den Raycaster

class Raycaster:
    @staticmethod
    def raycast_horizontal(
        maze, cell_width: int, rect: pygame.FRect, angle: float
    ) -> tuple[float, float, float, bool]:
        """
        Raycasting in horizontaler Richtung (über Zellen hinweg), um die erste Wand oder das Ziel zu treffen.
        :return: (Ray-Koordinaten, Ray-Länge, ob das Ziel erreicht wurde).
        """
        ray_x = 0
        ray_y = 0
        off_x = 0
        off_y = 0
        goal = False

        maze_size = len(maze)
        maze_width = maze_size * cell_width

        if angle == 0:
            return (0, 0, maze_width, goal)
        inverse_tan = 1 / math.tan(angle)

        if angle > math.pi:  # forward
            ray_y = math.floor(rect.centery / cell_width) * cell_width - 0.0001
            ray_sin = rect.centery - ray_y
            ray_cos = inverse_tan * ray_sin
            ray_x = rect.centerx - ray_cos
            off_y = -cell_width
            off_x = off_y * inverse_tan
        elif angle < math.pi:  # backward
            ray_y = math.ceil(rect.centery / cell_width) * cell_width
            ray_sin = rect.centery - ray_y
            ray_cos = inverse_tan * ray_sin
            ray_x = rect.centerx - ray_cos
            off_y = cell_width
            off_x = off_y * inverse_tan
        else:  # left or right
            return (0, 0, maze_width, goal)

        depth = maze_size
        while depth:
            mapx: int = int(ray_x / cell_width)
            mapy: int = int((ray_y / cell_width))
            inside_maze = (mapx < maze_size) and (mapx >= 0)
            if not inside_maze:
                return (0, 0, maze_width, goal)
            wall = maze[mapy][mapx]
            if wall:
                if wall == 2:
                    goal = True
                ray_len_x = rect.centerx - ray_x
                ray_len_y = rect.centery - ray_y
                ray_len = math.sqrt(math.pow(ray_len_x, 2) + math.pow(ray_len_y, 2))
                return (ray_x, ray_y, ray_len, goal)
            ray_x += off_x
            ray_y += off_y
            depth -= 1
        return (0, 0, maze_width, goal)

    @staticmethod
    def raycast_vertical(
        maze, cell_width: int, rect: pygame.FRect, angle: float
    ) -> tuple[float, float, float, bool]:
        """
        Raycasting in vertikaler Richtung (über Zellen hinweg), um die erste Wand oder das Ziel zu treffen.
        :return: (Ray-Koordinaten, Ray-Länge, ob das Ziel erreicht wurde).
        """
        ray_x = 0
        ray_y = 0
        off_x = 0
        off_y = 0
        goal = False

        maze_size = len(maze)
        maze_width = maze_size * cell_width

        if (angle > math.pi / 2) and (angle < 3 * math.pi / 2):  # left
            ray_x = math.floor(rect.centerx / cell_width) * cell_width - 0.0001
            ray_cos = rect.centerx - ray_x
            ray_sin = math.tan(angle) * ray_cos
            ray_y = rect.centery - ray_sin
            off_x = -cell_width
            off_y = off_x * math.tan(angle)
        elif (angle < math.pi / 2) or (angle > 3 * math.pi / 2):
            ray_x = math.ceil(rect.centerx / cell_width) * cell_width
            ray_cos = rect.centerx - ray_x
            ray_sin = math.tan(angle) * ray_cos
            ray_y = rect.centery - ray_sin
            off_x = cell_width
            off_y = off_x * math.tan(angle)
        else:  # forward or backwards
            return (0, 0, maze_width, goal)

        depth = maze_size
        while depth:
            mapx: int = int(ray_x / cell_width)
            mapy: int = int((ray_y / cell_width))
            inside_maze = (mapy < maze_size) and (mapy >= 0)
            if not inside_maze:
                return (0, 0, maze_width, goal)
            wall = maze[mapy][mapx]
            if wall:
                if wall == 2:
                    goal = True
                ray_len_x = rect.centerx - ray_x
                ray_len_y = rect.centery - ray_y
                ray_len = math.sqrt(math.pow(ray_len_x, 2) + math.pow(ray_len_y, 2))
                return (ray_x, ray_y, ray_len, goal)

            ray_x += off_x
            ray_y += off_y
            depth -= 1
        return (0, 0, maze_width, goal)

    @staticmethod
    def raycast(maze, rect: pygame.FRect, angle: float):
        """
        Führt sowohl horizontales als auch vertikales Raycasting durch und gibt den kürzeren Ray zurück.
        """
        ray_h = Raycaster.raycast_horizontal(maze.maze, maze.cell_width, rect, angle)
        ray_v = Raycaster.raycast_vertical(maze.maze, maze.cell_width, rect, angle)
        ray = min(ray_h, ray_v, key=lambda ray: ray[2])
        return ray

    @staticmethod
    def raycasting(
        maze,
        rect: pygame.FRect,
        player_angle: float,
        fov: int,
        amount: int,
    ):
        """
        Führt Raycasting für mehrere Strahlen durch (für das Sichtfeld des Spielers und der KI).
        :return: Eine Liste von Strahlen mit deren Koordinaten und Längen
        """
        fov_rad = math.radians(fov)
        fov_step = fov_rad / amount
        rays: list[tuple[float, float, float, float, bool]] = []
        for current_step in range(amount):
            angle = player_angle - fov_rad / 2 + fov_step * current_step
            if angle < 0:
                angle += 2 * math.pi
            if angle > 2 * math.pi:
                angle -= 2 * math.pi
            ray = Raycaster.raycast(maze, rect, angle)
            ray_length = ray[2]
            if ray[3] == True:
                ray_length = maze.image.width
            no_fish_angle = player_angle - angle
            if no_fish_angle < 0:
                no_fish_angle += 2 * math.pi
            if no_fish_angle > 2 * math.pi:
                no_fish_angle -= 2 * math.pi
            no_fish_length = math.cos(no_fish_angle) * ray[2]
            rays.append((ray[0], ray[1], ray_length, no_fish_length, ray[3]))
        return rays

In [ ]:
# Code für Player Klasse

import neat
from pygame import rect, key
import pygame
import math


class Player:
    speed = 4
    fov = 180
    rays_amount = 6
    LIFE_TIME = 40

    def __init__(
        self,
        pos: tuple[int, int],
        radius: int,
        boxes: list[pygame.Rect],
        boxes_type: list[bool],
        path_cells: list[pygame.Rect],
        genome,
        net: neat.nn.FeedForwardNetwork,
        maze_width: int,
        cell_width: int,
        best_genome: bool,
    ) -> None:
        self.image = pygame.Surface((radius * 2, radius * 2))
        self.rect: rect.FRect = self.image.get_frect(center=pos)
        self.boxes = boxes
        self.boxes_type = boxes_type
        self.path_cells = path_cells
        self.path_cells_score: list[int] = [0] * len(self.path_cells)
        self.genome = genome
        self.net = net
        self.best_genome = best_genome
        self.life_time = self.LIFE_TIME
        self.total_time = 0

        self.maze_width = maze_width
        self.cell_width = cell_width
        self.maze_inside_width = maze_width - cell_width * 2

        self.image.set_colorkey((0, 0, 0))
        pygame.draw.circle(self.image, "Red", (radius, radius), radius)

        self.direction = pygame.Vector2()
        self.angle = 0
        self.angle_direction = pygame.Vector2()
        self.rays: list[tuple[float, float, float, float, bool]] = []

    def wasd_input(self):
        keys = key.get_pressed()
        self.direction.x = int(keys[pygame.K_d]) - int(keys[pygame.K_a])
        self.direction.y = int(keys[pygame.K_s]) - int(keys[pygame.K_w])
        if self.direction:
            self.direction = self.direction.normalize()

    def angle_input(self):
        keys = key.get_pressed()
        # self.get_ai_input_data()
        self.angle += (keys[pygame.K_d] - keys[pygame.K_a]) / 10
        if self.angle < 0:
            self.angle += 2 * math.pi
        if self.angle > 2 * math.pi:
            self.angle -= 2 * math.pi
        direction = keys[pygame.K_w] - keys[pygame.K_s]
        self.angle_direction.x = math.cos(self.angle)
        self.angle_direction.y = math.sin(self.angle)
        self.direction.x = self.angle_direction.x * direction
        self.direction.y = self.angle_direction.y * direction

    def get_ai_input_data(self):
        inputs = []
        for ray in self.rays:
            normalised_ray = ray[2] / (
                self.maze_inside_width
            )  # when diagonal, potential to still be above 1
            inputs.append(normalised_ray)
        return inputs

    def ai_input(self):
        inputs = self.get_ai_input_data()
        output = self.net.activate(inputs)
        self.angle -= 0.1  # left
        if max(output[0], 0):  # if below 0 then 0
            self.angle += 0.2  # right
        direction = 0
        if max(output[1], 0):  # if below 0 then 0
            direction = 1  # forward
        if self.angle < 0:
            self.angle += 2 * math.pi
        if self.angle > 2 * math.pi:
            self.angle -= 2 * math.pi
        self.angle_direction.x = math.cos(self.angle)
        self.angle_direction.y = math.sin(self.angle)
        self.direction.x = self.angle_direction.x * direction
        self.direction.y = self.angle_direction.y * direction

    def raycasting(self, maze):
        self.rays = Raycaster.raycasting(
            maze, self.rect, self.angle, self.fov, self.rays_amount
        )

    def move(self):
        self.rect.x += self.direction.x * self.speed
        self.rect.y += self.direction.y * self.speed
        self.collision()

    def collision(self):
        collision_index = self.rect.collidelist(self.boxes)
        if collision_index != -1:
            is_goal = self.boxes_type[collision_index]
            # Tod nach Berührung mit Zelle. Belohnung falls Ziel und Bestrafung falls Wand.
            if is_goal:
                self.genome.fitness += 10000
                self.genome.fitness -= self.total_time
            else:
                self.genome.fitness -= 65
            self.life_time = 0
            return

    def path_collision(self):
        collision_index = self.rect.collidelist(self.path_cells)
        if collision_index != -1:
            if self.path_cells_score[collision_index] == 0:  # new cell
                self.life_time += 20
                self.genome.fitness += 8
            self.path_cells_score[collision_index] += 1
            if self.path_cells_score[collision_index] > 50:
                self.life_time = 0
                self.genome.fitness -= 80

    def update(self, maze):
        self.raycasting(maze)
        self.ai_input()
        self.move()
        self.path_collision()
        self.life_time -= 1
        self.total_time += 1

    def draw_rays(self, screen: pygame.Surface):
        for ray in self.rays:
            if ray[4]:
                pygame.draw.line(screen, "Blue", self.rect.center, (ray[0], ray[1]), 2)
            else:
                pygame.draw.line(screen, "Green", self.rect.center, (ray[0], ray[1]), 2)

    def draw_look_direction(self, screen: pygame.Surface):
        end_line_x = self.rect.centerx + self.angle_direction.x * 20
        end_line_y = self.rect.centery + self.angle_direction.y * 20
        pygame.draw.line(screen, "Blue", self.rect.center, (end_line_x, end_line_y), 2)

    def draw(self, screen: pygame.Surface, maze):
        self.draw_look_direction(screen)
        if self.best_genome:
            self.draw_rays(screen)
            self.draw_3D(screen, maze)
        screen.blit(self.image, self.rect)

    def draw_3D(self, screen: pygame.Surface, maze):
        line_width = int(self.maze_width / self.rays_amount / 40)
        current_x = self.maze_width + line_width / 2
        rays = Raycaster.raycasting(
            maze, self.rect, self.angle, 90, self.rays_amount * 40
        )
        for ray in rays:
            length = self.maze_width / ray[3] * self.cell_width
            length = min(length, self.maze_width)
            if ray[4]:
                pygame.draw.line(
                    screen,
                    "Blue",
                    (current_x, self.maze_width / 2 - length / 2),
                    (current_x, self.maze_width / 2 + length / 2),
                    line_width,
                )
            else:
                pygame.draw.line(
                    screen,
                    "Green",
                    (current_x, self.maze_width / 2 - length / 2),
                    (current_x, self.maze_width / 2 + length / 2),
                    line_width,
                )
            current_x += line_width

    def ai_view(self, screen: pygame.Surface, maze):
        line_width = int(self.maze_width / 6)
        current_x = self.maze_width + line_width / 2
        rays = Raycaster.raycasting(
            maze, self.rect, self.angle, 90, 6
        )
        for ray in rays:
            length = self.maze_width / ray[2] * self.cell_width
            length = min(length, self.maze_width)
              # Calculate brightness (closer = brighter, farther = darker)

            brightness = max(0, min(255, int(255 / (1 + ray[3] * 0.05))))
            
            if ray[4]:  # If hit a wall
                color = (0, 0, brightness)  # Blue with darkness effect
            else:
                color = (0, brightness, 0)  # Green with darkness effect
            pygame.draw.line(
                screen,
                color,
                (current_x, 0),
                (current_x, self.maze_width),
                line_width,
            )
            current_x += line_width

In [ ]:
# Code für Game Klasse

import neat
import pygame


class Game:
    TOTAL_WIDTH = 960
    TOTAL_HEIGHT = 720
    FPS = 60

    def __init__(self, max_rounds):
        pygame.init()
        self.maze = MazeRendererWithCollision(10, 40)
        self.screen = pygame.display.set_mode(
            (self.maze.image.width * 2, self.maze.image.height)
        )
        self.clock = pygame.time.Clock()
        self.running = True

        self.players: list[game.Player] = []
        self.ticks = 0
        self.round = 0
        self.max_rounds = max_rounds

    def setup(self, genomes, config, best_genome):
        if self.round > self.max_rounds:
            self.maze = MazeRendererWithCollision(10, 40)
            self.round = 0
            if self.max_rounds > 0:
                self.max_rounds -= 0.5
        posx = int(self.maze.cell_width * 1.5)
        best_genome_id = best_genome[0]
        for i, genome in genomes:
            best_genome = False
            if i == best_genome_id:
                best_genome = True
            net = neat.nn.FeedForwardNetwork.create(genome, config)
            player = Player(
                (posx, posx),
                5,
                self.maze.boxes,
                self.maze.boxes_type,
                self.maze.path_cells,
                genome,
                net,
                self.maze.image.width,
                self.maze.cell_width,
                best_genome,
            )
            self.players.append(player)

    def update(self):
        for i, player in enumerate(self.players):
            player.update(self.maze)
            if player.life_time <= 0:
                # print("fitness:", player.genome.fitness)
                del self.players[i]
        self.ticks += 1

    def draw(self):
        self.screen.fill("Black")
        self.maze.draw(self.screen)
        for player in self.players:
            player.draw(self.screen, self.maze)
        pygame.display.update()

In [ ]:
# Code für Main Klasse

from typing import Any
import pygame
import os
import neat
import pickle


def eval_genomes(genomes: list[Any], config, game: Game, render: bool):
    def best_genome_max(genome): # Used for rendering while training
        if genome[1].fitness == None:
            return -100
        return genome[1].fitness

    best_genome = max(genomes, key=best_genome_max)
    for _, genome in genomes:
        genome.fitness = 0
    game.setup(genomes, config, best_genome)
    while len(game.players):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                game.running = False
        game.update()
        if render:
            game.draw()
            game.clock.tick(60)
    game.round += 1
    


def train_ai(config_file: str, game: Game, n_gen: int, render: bool, checkpoint: str):
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )
    # Create the population, which is the top-level object for a NEAT run.
    p = neat.Population(config)
    if checkpoint:
        p = neat.Checkpointer.restore_checkpoint(f"checkpoints/{checkpoint}")

    # Add a stdout reporter to show progress in the terminal.
    p.add_reporter(neat.StdOutReporter(True))
    stats = neat.StatisticsReporter()
    p.add_reporter(stats)
    checkpointer = neat.Checkpointer(generation_interval=5, filename_prefix="checkpoints/neat-checkpoint-")
    p.add_reporter(checkpointer)

    # Damit zusätzliche Parameter mitgegeben werden können
    def execute_eval_genomes_func(genomes, config):
        eval_genomes(genomes, config, game, render)

    # Run for up to "n_gen" amount of generations.
    winner = p.run(execute_eval_genomes_func, n_gen)
    with open("best.pickle", "wb") as f:
        pickle.dump(winner, f)


def test_ai(config_file: str, game: Game):
    config = neat.Config(
        neat.DefaultGenome,
        neat.DefaultReproduction,
        neat.DefaultSpeciesSet,
        neat.DefaultStagnation,
        config_file,
    )
    with open("best.pickle", "rb") as f:
        winner = pickle.load(f)
    while game.running:
        game.setup([[0, winner]], config, [0])
        while len(game.players):
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    game.running = False
            game.update()
            game.draw()
            game.clock.tick(60)
        game.round += 26

def execute_train_test(mode, n_gen=0, render=False, checkpoint=None, max_rounds=2):
    if mode == "test":
        game = Game(max_rounds=2)
        test_ai("config.txt", game)
    else:
        game = Game(max_rounds)
        train_ai("config.txt", game, n_gen, render, checkpoint)
    pygame.quit()

In [ ]:
import ipywidgets as widgets
import re

# Directory where checkpoints are stored
checkpoint_dir = "checkpoints"

os.makedirs(checkpoint_dir, exist_ok=True)

def extract_gen_number(filename):
    match = re.search(r"neat-checkpoint-(\d+)", filename)
    return int(match.group(1)) if match else -1

# Get sorted list of checkpoint files by generation number
def get_checkpoints():
    checkpoints = [f for f in os.listdir(checkpoint_dir) if f.startswith("neat-checkpoint-")]
    return sorted(checkpoints, key=extract_gen_number)

# Function to reload the checkpoint list
def reload_checkpoints(b):
    checkpoint_selector.options = get_checkpoints()

# Function to update visibility based on mode
def update_visibility(change):
    if mode_dropdown.value == "train":
        checkpoint_selector.layout.display = 'none'
        reload_button.layout.display = 'none'
        clear_button.layout.display = "none"

        n_gen_number_text.layout.display = "block"
        render_bool.layout.display = "block"
    
    elif mode_dropdown.value == "train-checkpoint":
        checkpoint_selector.layout.display = 'block'
        reload_button.layout.display = 'block'
        clear_button.layout.display = "block"

        n_gen_number_text.layout.display = "block"
        render_bool.layout.display = "block"
    else:
        checkpoint_selector.layout.display = 'none'
        reload_button.layout.display = 'none'
        clear_button.layout.display = "none"

        n_gen_number_text.layout.display = "none"
        render_bool.layout.display = "none"

checkpoint_selector = widgets.Dropdown(
    options=get_checkpoints(),
    description="Checkpoint:",
    disabled=False
)

clear_button = widgets.Button(description="Clear Checkpoints")
clear_button.layout.display = 'none'

def clear_checkpoints(b):
    with output:
        output.clear_output()
        for f in os.listdir(checkpoint_dir):
            if f.startswith("neat-checkpoint-"):
                os.remove(os.path.join(checkpoint_dir, f))
        print("All checkpoints cleared.")
        reload_checkpoints(None)  # Refresh dropdown
            

clear_button.on_click(clear_checkpoints)

checkpoint_selector.layout.display = 'none'  # Hide initially
# Button to reload checkpoint list (hidden by default)
reload_button = widgets.Button(description="Reload Checkpoints")
reload_button.layout.display = 'none'  # Hide initially
reload_button.on_click(reload_checkpoints)

In [ ]:
mode_dropdown = widgets.Dropdown(
    options=[("Trainieren", "train"),("Testen", "test"), ("Trainieren mit Checkpoint","train-checkpoint")],
      value="train",
    description='Mode:',
)

mode_dropdown.observe(update_visibility, names='value')

start_button = widgets.Button(description="Start")

quit_button = widgets.Button(description="Quit")

output = widgets.Output()
n_gen_number_text = widgets.IntText(
    value=200,
    description='Anzahl an generationen:',
    disabled=False
)
n_gen_number_text = widgets.IntText(
    value=200,
    description='Anzahl an generationen:',
    disabled=False
)
n_max_rounds_widget = widgets.IntText(
    value=25,
    description='Anzahl an maximalen Runden:',
    disabled=False
)
render_bool = widgets.Checkbox(
    value=False,
    description='Render',
    disabled=False,
    indent=False
)

def on_start_button_clicked(d):
    with output:
        output.clear_output()
        mode = mode_dropdown.value
        n_gen = n_gen_number_text.value
        render = render_bool.value
        checkpoint = None
        n_max_rounds = n_max_rounds_widget.value
        if mode_dropdown.value == "train-checkpoint":
            checkpoint = checkpoint_selector.value
        execute_train_test(mode_dropdown.value, n_gen, render, checkpoint, n_max_rounds)

def quit_game(d):
    pygame.quit()

start_button.on_click(on_start_button_clicked)

quit_button.on_click(quit_game)


display(mode_dropdown, checkpoint_selector, reload_button, clear_button, n_gen_number_text, n_max_rounds_widget, render_bool, start_button, quit_button, output)

In [ ]:
import nbformat
from IPython.display import Markdown, display

# Specify the tags you want to exclude.
excluded_tags = {"skip-export"}

with open("neat.ipynb", "r", encoding="utf-8") as f:
    nb = nbformat.read(f, as_version=4)

all_code = "```python\n"

for cell in nb.cells:
    if cell.cell_type == "code":
        # Get the tags from the cell's metadata (if any)
        tags = cell.get("metadata", {}).get("tags", [])
        # Skip the cell if it has any tag from the excluded set
        if any(tag in excluded_tags for tag in tags):
            continue
        all_code += cell.source + "\n\n"

all_code += "```"

display(Markdown(all_code))